In [11]:
import os
import sys
import subprocess
from contextlib import contextmanager
import signal
import filecmp
import difflib
import shutil
import time

# Configurations
toolchain_path = "/home/r1c0/riscv/bin/riscv64-unknown-linux-gnu-"
compiler_path = "./build/Compiler"
test_folder="./testcase"
sylib_path = "./RISCVLib/sylib.c"
# xxx.sy 会被复制到./function_test/test.c
temp_path = "./function_test"
pass_args=[
                "-S",
                "-o",
                "./function_test/test.s",
                "./function_test/test.c",
                "--O1"
            ]
Tolerance = True

CE_list = [] # Compile Error
AE_list = [] # Assembler Error
LE_list = [] # Linker Error
Time_Out = []
WA_list = []
AC_list = []
TLE_list = []
BadTest_list = []

Prob_list = []
test_list = []

In [12]:
# grab all the testcases
# find files recursively
def GrabFile():
    for root, dirs, files in os.walk(test_folder):
        for file in files:
            if file.endswith(".sy"):
                test_list.append(os.path.join(root, file))

In [13]:
def CopyFileToTemp(test):
    # No Bad Test Now
    shutil.copy(test,temp_path+"/test.c")
    if os.path.exists(test[:-2]+"in"):
        shutil.copy(test[:-2]+"in",temp_path+"/test.in")
    if os.path.exists(test[:-2]+"out"):
        shutil.copy(test[:-2]+"out",temp_path+"/test.out")

In [14]:
def TryCompile(original_case_path):
    compile_args=[compiler_path]
    for arg in pass_args:
        compile_args.append(arg)
    try:
        ret = subprocess.run(compile_args,timeout=60)
    except subprocess.TimeoutExpired:
        Time_Out.append(original_case_path)
        print("Timeout: "+original_case_path)
        return False
    if ret.returncode != 0:
        CE_list.append(original_case_path)
        print("CE: "+original_case_path)
        return False
    return True

In [15]:
def TryAssemble(original_case_path):
    try:
        ret = subprocess.run([toolchain_path+"as", temp_path + "/test.s", "-o", temp_path + "/test.o"],timeout=10)
    except subprocess.TimeoutExpired:
        print("IMPOSSIBLE, AS TIMEOUT")
        exit(-1)
    if ret.returncode != 0:
        AE_list.append(original_case_path)
        print("AE: "+original_case_path)
        return False
    return True

In [16]:
def TryLink(orginal_case_path):
    try:
        ret = subprocess.run([toolchain_path+"gcc", temp_path + "/test.o", temp_path+"/sylib.o", "-o", temp_path + "/test"],timeout=10)
    except subprocess.TimeoutExpired:
        print("IMPOSSIBLE, LD TIMEOUT")
        exit(-1)
    if ret.returncode != 0:
        LE_list.append(orginal_case_path)
        print("LE: "+orginal_case_path)
        return False
    return True

In [17]:
def RunTest(original_case_path, test_args):
    start_time=time.time()
    try:
        if not os.path.exists(temp_path+"/test.in"):
            ret = subprocess.run(test_args,stdout=subprocess.PIPE,stderr=subprocess.PIPE,timeout=30)
        else:
            ret = subprocess.run(test_args,stdin=open(temp_path+"/test.in"),stdout=subprocess.PIPE,stderr=subprocess.PIPE,timeout=30)
    except subprocess.TimeoutExpired:
        print("TIMEOUT ERROR: "+original_case_path)
        TLE_list.append(original_case_path)
        return False
    end_time=time.time()
    
    elasp_time=end_time-start_time
    
    if not os.path.exists(temp_path + "/test.out"):
        BadTest_list.append(original_case_path)
        return False
    else:
        out_file=temp_path + "/test.out"
    
    dump_str=ret.stdout.decode()
    # dump_str1 = ret1.stdout.decode()
    # remove whitesspace in the end
    # dump_str=dump_str.rstrip()
    # if dump_str1 and not dump_str1.endswith('\n'):
    #     dump_str1 += "\n"
    # if not dump_str1.endswith(''):
    #     print("RISCV Test Error")
    if dump_str and not dump_str.endswith('\n'):
        dump_str += "\n"
    dump_str += str(ret.returncode) + "\n"
    std_output=open(out_file).read()
    diff = difflib.unified_diff(dump_str.splitlines(), std_output.splitlines(), lineterm='')
    if(len(list(diff))!=0):
        print("Wrong Answer: "+original_case_path)
        WA_list.append(original_case_path)
        return False
    
    print(f"{elasp_time:.2f}---",end="")  # Print runtime information
    return True

def LLI_TEST(original_case_path):
    print("LLI\t",end=":")
    if RunTest(original_case_path,["/home/r1c0/llvm-project/build/bin/lli",temp_path + "/test.c.ll"]):
        print("Pass:"+original_case_path)
        return True
    else:
        return False

def QEMU_TEST(original_case_path):
    print("QEMU\t",end=":")
    if RunTest(original_case_path,["qemu-riscv64",temp_path + "/test"]):
        print("Pass:"+original_case_path)
        return True
    else:
        return False

In [18]:
def Tolerant(FuncList,original_file_name):
    for func in FuncList:
        ret = func(original_file_name)
        if ret == False and Tolerance == False:
            print("No Tolerence, exit")
            sys.exit(-1)
        if ret == False:
            return False
    return True

In [19]:
def TestSingleCase(test):
    CopyFileToTemp(test)
    if Tolerant([TryCompile,TryAssemble,TryLink,QEMU_TEST],test)==False:
        return
    AC_list.append(test)

In [20]:
# Compile the sylib.c
subprocess.run([toolchain_path+"gcc", sylib_path, "-c", "-o", temp_path+"/sylib.o"])

FileNotFoundError: [Errno 2] No such file or directory: '/home/r1c0/riscv/bin/riscv64-unknown-linux-gnu-gcc'

In [ ]:
# if len(Prob_list) == 0:
#     GrabFile()
# else:
#     test_list=Prob_list
#     Prob_list.clear()

test_list=['./testcases/performance/01_mm1.sy']
    
for test in test_list:
    if test.endswith(".sy"):
        TestSingleCase(test)

In [ ]:
# print all wrong tests and mv one of them to temp_folder for debugging
print(len(AC_list))
print(len(WA_list))
print(len(CE_list))
print(len(AE_list))
print(len(LE_list))
print(len(Time_Out))
print(len(TLE_list))
print(len(BadTest_list))

In [ ]:
def MV2WorkSpace(CasesList,k):
    case = CasesList[k]
    CopyFileToTemp(case)
    print(case)
    return
        
# MV2WorkSpace(TLE_list,0)
# TLE_list=['./testcases/h_performance/h-2-03.sy', './testcases/h_performance/h-3-01.sy', './testcases/h_performance/h-2-02.sy', './testcases/h_performance/h-3-03.sy', './testcases/h_performance/h-2-01.sy', './testcases/h_performance/h-3-02.sy', './testcases/performance/gameoflife-gosper.sy', './testcases/performance/if-combine1.sy', './testcases/performance/if-combine3.sy', './testcases/performance/matmul2.sy', './testcases/performance/if-combine2.sy', './testcases/performance/matmul3.sy', './testcases/performance/large_loop_array_1.sy', './testcases/performance/gameoflife-p61glidergun.sy', './testcases/performance/large_loop_array_2.sy', './testcases/performance/recursive_call_1.sy', './testcases/performance/large_loop_array_3.sy']

# MV2WorkSpace(TLE_list,0)

print(WA_list)
print(CE_list)
print(Time_Out)
print(TLE_list)